In [11]:
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk
import torch
from transformers import AutoImageProcessor
import joblib
import pandas as pd

# Load LabelEncoder and class names
label_encoder = joblib.load("label_encoder.pkl")
class_names = list(label_encoder.classes_)

# Initialize the image processor
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")

# Load the trained model
model = torch.load("model_imp_v2.pth", map_location=torch.device("cpu"))
model.eval()

# Function to process and predict
def predict_image(filepath):
    # Load and preprocess the image
    image = Image.open(filepath).convert("RGB")
    inputs = image_processor(image, return_tensors="pt")
    
    # Predict using the model
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits[0], dim=0)
        top2_prob, top2_idx = torch.topk(probabilities, 2)

    # Map indices to breed names
    try:
        top2_breeds = [(class_names[idx], prob.item()) for idx, prob in zip(top2_idx, top2_prob)]
    except IndexError as e:
        print(f"Error: {e}")
        return [("Unknown", 0.0), ("Unknown", 0.0)]
    
    return top2_breeds

# Function to open file dialog
def open_file_dialog():
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
    if filepath:
        # Display the image
        image = Image.open(filepath).resize((250, 250))
        img = ImageTk.PhotoImage(image)
        img_label.configure(image=img)
        img_label.image = img

        # Predict and display results
        predictions = predict_image(filepath)
        result_text = (f"Your dog is a {predictions[0][0]} with {predictions[0][1]:.2%} probability "
                       f"and a {predictions[1][0]} with {predictions[1][1]:.2%} probability.")
        result_label.configure(text=result_text)
    else:
        result_label.configure(text="Please upload a dog picture!")

# Create GUI window
window = tk.Tk()
window.title("Dog Breed Predictor")

# Add prompt label
prompt_label = Label(window, text="Please upload a dog picture!", font=("Helvetica", 16), fg="blue")
prompt_label.pack()

# Add upload button
upload_button = tk.Button(window, text="Upload Image", command=open_file_dialog)
upload_button.pack()

# Add image display label
img_label = Label(window)
img_label.pack()

# Add result display label
result_label = Label(window, text="", font=("Helvetica", 14))
result_label.pack()

# Run the GUI event loop
window.mainloop()


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
